# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [4]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [8]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [10]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'portfolio', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'portfolio', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/11/11/ai-live-event/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
 

In [11]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [12]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 3 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'}]}

In [13]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'endpoints page', 'url': 'https://endpoints.huggingface.co'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Discord', 'url': 'https://huggingface.co/join/discord'},
  {'type': 'Blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'Learn', 'url': 'https://huggingface.co/learn'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [14]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [15]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 16 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Tongyi-MAI/Z-Image-Turbo
Updated
10 days ago
•
323k
•
3k
nvidia/NVIDIA-Nemotron-3-Nano-30B-A3B-BF16
Updated
about 4 hours ago
•
51.3k
•
361
tencent/HY-WorldPlay
Updated
about 10 hours ago
•
314
XiaomiMiMo/MiMo-V2-Flash
Updated
about 23 hours ago
•
3.12k
•
281
ResembleAI/chatterbox-turbo
Updated
3 days ago
•
261
Browse 1M+ models
Spaces
Running
on
Zero
MCP
Featured
275
Chatterbox Turbo Demo
⚡
275
Chatterbox Turbo Demo
Running
on
Zero
254
TRELLIS.2
🏢
254
High-fidelity 3D Generation from images
Running
on
Zero
681
Z Image Turbo
🖼
681
Ge

In [16]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 25 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTongyi-MAI/Z-Image-Turbo\nUpdated\n10 days ago\n•\n323k\n•\n3k\nnvidia/NVIDIA-Nemotron-3-Nano-30B-A3B-BF16\nUpdated\nabout 4 hours ago\n•\n51.3k\n•\n361\ntencent/HY-WorldPlay\nUpdated\nabout 10 hours ago\n•\n314\nXiaomiMiMo/MiMo-V2-Flash\nUpdated\nabout 23 hours ago\n•\n3.12k\n•\n281\nResembleAI/chatterbox-turbo\nUpdated\n3 days ago\n•\n261\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\nMCP

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


# Hugging Face Brochure

---

## Who We Are  
Hugging Face is the AI community building the future of machine learning. At our core, we are a vibrant platform where practitioners, researchers, and organizations collaborate on developing, sharing, and improving AI models, datasets, and applications. We empower the global machine learning community to create better AI faster in an open, collaborative environment.

---

## Our Platform  
- **Models:** Access and browse over **1 million AI models** spanning multiple modalities including text, image, video, audio, and 3D.  
- **Datasets:** Discover and collaborate on **250,000+ datasets** from diverse domains to fuel your machine learning projects.  
- **Spaces:** Host, share, and interact with cutting-edge AI applications and demos seamlessly on our platform.  
- **Community:** Engage with an active and growing ecosystem that shares knowledge, resources, and best practices in ML.

We provide an open-source stack to help you move faster, build your portfolio, and accelerate your machine learning journey.

---

## Products & Solutions

### Open Source & Free Access  
- Host unlimited public projects for models, datasets, and apps.  
- Explore trending projects and contribute to a vast catalog of AI resources.  

### Paid Compute and Enterprise Solutions  
- **Team Plan:** Start at $20/user/month, ideal for scaling collaboration within your teams.  
- **Enterprise Plan:** Customized plans with enterprise-grade security, access controls, dedicated support, Single Sign-On (SSO) integration, regional data management, and audit logs.  

Our enterprise offering ensures secure, scalable AI development workflows tailored to organizational needs.

---

## Company Culture  
- **Community-Driven:** Collaboration is at the heart of everything we do. We foster a culture where knowledge sharing accelerates collective progress.  
- **Open Innovation:** We prioritize openness and transparency through open source, empowering the democratization of AI technology.  
- **Diversity of Modalities:** We encourage exploration across AI modalities—text, image, video, audio, and more—welcoming innovators from all backgrounds.  
- **Growth & Learning:** Our platform is designed to help practitioners build their portfolio and reputations within the ML community worldwide.

---

## Who Uses Hugging Face?  
- **AI Researchers & Practitioners:** Develop, share, and benchmark models and datasets.  
- **Startups & Enterprises:** Leverage scalable AI infrastructure and security controls to integrate AI solutions rapidly.  
- **Developers & Data Scientists:** Access ready-to-use models and datasets to jumpstart projects and collaborate with peers.  
- **Academic Institutions:** Promote AI research and teaching with accessible tools and shared resources.

---

## Careers

Hugging Face offers exciting opportunities for talent passionate about the future of AI. Join us to work alongside industry leaders in a collaborative environment that values innovation, diversity, and open-source contribution.

Check our website for the latest openings in engineering, research, product management, and customer success.

---

## Why Choose Hugging Face?  
- Leading platform with the largest collection of AI models and datasets.  
- Deeply engaged and growing AI community.  
- Industry-grade enterprise security and support.  
- Seamless experience combining open source with professional solutions.  
- Commitment to accelerate the development and application of trustworthy AI.

---

## Get Started Today  
- Explore millions of models and datasets at [huggingface.co](https://huggingface.co)  
- Join the AI community shaping the future.  
- Scale your AI projects with flexible Team and Enterprise plans.  

**Hugging Face — The AI Community Building the Future**

---

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is the AI community building the future of machine learning. As a vibrant collaboration platform, it brings together machine learning engineers, scientists, and enthusiasts from all over the world to co-create, explore, and share models, datasets, and AI applications.

Serving as the home of machine learning, Hugging Face empowers its global community to build and experiment with cutting-edge machine learning technologies across multiple modalities including text, image, video, audio, and even 3D.

---

## What We Offer

- **Collaborative Platform:**  
  Host and collaborate on unlimited public models, datasets, and applications. Hugging Face's platform supports seamless sharing and discovery of ML resources.

- **Huge Repository:**  
  Access over **1 million** machine learning models and **250,000+** datasets to accelerate your AI projects.

- **Spaces:**  
  Showcase and explore thousands of AI-powered applications built on Hugging Face's open ecosystem, including featured demos like image generation and 3D modeling.

- **Open Source Stack:**  
  Leverage our open source tools and frameworks to move faster and innovate in your AI development lifecycle.

- **Enterprise Solutions:**  
  Benefit from advanced paid Compute and enterprise-level solutions designed to empower teams and organizations to build AI at scale securely and efficiently.

---

## Community & Culture

Hugging Face fosters an open, collaborative, and ethical AI culture, encouraging the sharing of knowledge and resources to drive innovation. The platform serves as a central hub where anyone—from students to experts—can contribute to and benefit from a fast-growing ecosystem of machine learning projects.

The company values transparency, inclusivity, and continuous learning, nurturing a community that is passionate about creating an ethical AI future together.

---

## For Customers and Developers

- Explore and integrate state-of-the-art models and applications to enhance your AI solutions.
- Collaborate seamlessly with peers and experts worldwide.
- Build your personal ML portfolio and showcase your contributions to the global community.
- Access cutting-edge datasets across multiple domains for research and development.
- Utilize enterprise-level compute resources for scalable and secure machine learning deployments.

---

## Careers at Hugging Face

Joining Hugging Face means becoming part of a visionary team dedicated to democratizing AI. The company seeks talented individuals who are passionate about open source, innovation, and making a positive impact on the future of AI.

Positions span across engineering, research, community development, and enterprise support, offering opportunities to contribute to a thriving global AI community while working with state-of-the-art technologies.

---

## Contact & Get Involved

- **Website:** [huggingface.co](https://huggingface.co)  
- **Community:** Join the platform to start collaborating, sharing, and building AI with peers worldwide.  
- **Explore Models & Datasets:** Discover millions of resources available free to the public.  
- **Enterprise Solutions:** Contact Hugging Face sales for tailored compute and team collaboration offerings.

---

## Brand Identity

- **Colors:**  
  - Yellow (#FFD21E)  
  - Orange (#FF9D00)  
  - Gray (#6B7280)  

- **Logo:** Available in multiple formats including SVG, PNG, AI for versatile use.

Hugging Face’s friendly and accessible brand reflects its commitment to an open, ethical, and collaborative AI future.

---

*Build, share, and innovate in machine learning with Hugging Face — the AI community building the future.*

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>